In [ ]:
# aind-data-schema              0.38.0

In [1]:
!pip install matplotlib
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 61.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 100.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 75.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 47.6 MB/s eta 0:00:00


In [31]:
import glob
import os
from multiprocessing.pool import Pool, ThreadPool
from pathlib import Path
import pickle
from time import time

import h5py
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hdmf_zarr import NWBZarrIO

from aind_ophys_utils.signal_utils import median_filter
from scipy.ndimage import uniform_filter1d
from scipy.signal import butter, sosfilt
from sklearn.linear_model import LinearRegression
from statsmodels.api import RLM
from statsmodels.robust.norms import HuberT, TukeyBiweight

import utils.nwb_dict_utils as nwb_utils
from utils.preprocess import batch_processing

In [3]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.options.mode.copy_on_write = True

In [4]:
def RMSE(x,y):
    return np.sqrt(np.mean((x-y)**2, -1))

def bias(x,y):
    return np.mean(x-y, -1)

def std_of_diff(x,y):
    return np.std(x-y, -1)

def corr(x, y):
    x = x-x.mean(-1)[...,None]
    x /= x.std(-1)[...,None]
    y = y-y.mean(-1)[...,None]
    y /= y.std(-1)[...,None]
    return np.mean(x*y, -1)

In [5]:
parents = sorted(glob.glob("/data/photometry/vary*"))
parents

['/data/photometry/vary_a_max',
 '/data/photometry/vary_a_power',
 '/data/photometry/vary_attenuation',
 '/data/photometry/vary_b_bright',
 '/data/photometry/vary_b_fast',
 '/data/photometry/vary_b_inf',
 '/data/photometry/vary_b_slow',
 '/data/photometry/vary_corr_s',
 '/data/photometry/vary_decay',
 '/data/photometry/vary_motion_power',
 '/data/photometry/vary_noise_std',
 '/data/photometry/vary_t_bright',
 '/data/photometry/vary_t_fast',
 '/data/photometry/vary_t_slow']

In [5]:
# def evaluate(parent):
#     perf = []
#     params = []
#     dFFs = []
#     vary = parent.split('/')[-1][5:]
#     for nwb_file in sorted(glob.glob(f"{parent}/*/*.nwb")):
#         print(f"Processing NWB file: {nwb_file}")
#         p = pd.read_csv(Path(nwb_file).parent / "parameters.csv", delimiter=",", index_col=0)
#         p = list(p[vary])
#         params.append([p[:3], p[3:6]])
#         with h5py.File(Path(nwb_file).parent / 'groundtruth.h5') as file:
#             dFF_gt = [file[f'fiber{f}']['dff'][:] for f in (0,1)]

#         with NWBZarrIO(path=nwb_file, mode="r") as io:
#             nwb = io.read()
#             # convert nwb to dataframe
#             df_from_nwb = nwb_utils.nwb_to_dataframe(nwb)
#             # add the session column
#             filename = Path(nwb_file).name
#             session_name = filename.split(".")[0].split("FIP_")[1]
#             df_from_nwb.insert(0, "session", session_name)
#             # now pass the dataframe through the preprocessing function:
#             df_fip_pp_nwb, df_PP_params = batch_processing(df_fip=df_from_nwb)
#             # calculate performance measures
#             tmp = []
#             dFF = []
#             for pre in ("poly", "exp", "bright"):
#                 df = df_fip_pp_nwb[df_fip_pp_nwb["preprocess"]==pre]
#                 dFF.append(np.array([[df[(df["channel"] == ch) & (df["fiber_number"] == fiber)]["signal"]
#                                  for ch in ['Iso', 'G', 'R']] for fiber in df_fip_pp_nwb["fiber_number"].unique()]))
#                 tmp.append([RMSE(dFF[-1], dFF_gt), bias(dFF[-1], dFF_gt), std_of_diff(dFF[-1], dFF_gt)])
#             perf.append(tmp)
#             dFFs.append(dFF)

#     return (np.array(params), # indices: expId, fiber, channel
#             np.array(perf),   # indices: expId, method, performance_measure, fiber, channel
#             np.array(dFFs))   # indices: expId, method, fiber, channel, frame

In [7]:
# %%time
# %%capture
# results = Pool(int(os.environ['CO_CPUS'])).map(evaluate, parents)

CPU times: user 3.02 s, sys: 3.31 s, total: 6.33 s
Wall time: 40min 47s


In [8]:
# # Save to a pickle file
# with open('/scratch/comparison.pkl', 'wb') as f:
#     pickle.dump(results, f)

In [6]:
# Load pickle file
with open('/scratch/comparison.pkl', 'rb') as f:
    results = pickle.load(f)

In [67]:
def motion_correct(dff, filt=lambda x: median_filter(x, 10), M=TukeyBiweight(1), filter_both=False, zero_mean=True):
    motion = dff[0]
    dff_filt = dff.copy()
    if filt is not None:
        motion = filt(motion)
        if filter_both:
            dff_filt = np.array(list(map(filt, dff)))
    if M is not None:
        motion = np.maximum([RLM(d, motion, M=M).fit().params for d in dff_filt], 0) * motion  # should we rather use robust regression?
    else:
        motion = LinearRegression(fit_intercept=False, positive=True
                                 ).fit(motion[:,None], dff_filt.T).predict(motion[:,None]).T
    dff_mc = dff - motion
    if zero_mean:
        dff_mc += motion.mean(-1)[...,None]
    return dff_mc, motion

In [68]:
def evaluate(row, filt=lambda x: x, M=TukeyBiweight(1), filter_both=False, zero_mean=True, pres=("bright",)):
    perf, perf_motion = [], []

    parent = parents[row]
    for exp_id, nwb_file in enumerate(sorted(glob.glob(f"{parent}/*/*.nwb"))):
        # print(f"Processing NWB file: {nwb_file}")
        with h5py.File(Path(nwb_file).parent / 'groundtruth.h5') as file:
            dFF_gt = [file[f'fiber{f}/dff'][:] for f in (0,1)]
            motion_gt = [file[f'fiber{f}/motion'][:] for f in (0,1)]
        tmp = []
        tmpm = []
        for pre in pres:
            dFFpre = results[row][2][exp_id, {"poly": 0, "exp": 1, "bright": 2}[pre]]
            dFF, motion = np.transpose([motion_correct(dff, filt, M, filter_both, zero_mean) for dff in dFFpre], (1,0,2,3))
            tmp.append([RMSE(dFF, dFF_gt), bias(dFF, dFF_gt), std_of_diff(dFF, dFF_gt)])
            tmpm.append(np.array([[np.corrcoef(m,g)[0,1] for m,g in zip(motion[f], motion_gt[f])] for f in (0,1)]))
        perf.append(tmp)
        perf_motion.append(tmpm)
    return perf, perf_motion

## look for a good M estimator

In [23]:
%%capture
N = len(parents)
Ms = [None, HuberT(), HuberT(1), HuberT(.5), HuberT(.25), HuberT(.1),
      TukeyBiweight(), TukeyBiweight(2), TukeyBiweight(1), TukeyBiweight(.5), TukeyBiweight(.25)]
keys = ["OLS", "HuberT()", "HuberT(1)", "HuberT(.5)", "HuberT(.25)", "HuberT(.1)",
        "TukeyBiweight()", "TukeyBiweight(2)", "TukeyBiweight(1.2)", "TukeyBiweight(1)", "TukeyBiweight(.8)", "TukeyBiweight(.5)", "TukeyBiweight(.25)"]
res = {}
for k, m in zip(keys, Ms):
    res[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [lambda x: x]*N, [m]*N))

In [49]:
print("Method       RMSE        bias          std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res[k]]).mean(0)[...,1:].mean((1,2)))

Method       RMSE        bias          std_of_diff
OLS [ 0.00842387 -0.00449494  0.00664619]
HuberT() [ 0.00821922 -0.00449494  0.00639203]
HuberT(1) [ 0.00816905 -0.00449494  0.006327  ]
HuberT(.5) [ 0.00810038 -0.00449494  0.00623643]
HuberT(.25) [ 0.00807805 -0.00449494  0.00620658]
HuberT(.1) [ 0.00807128 -0.00449494  0.00619747]
TukeyBiweight() [ 0.00816661 -0.00449494  0.0063263 ]
TukeyBiweight(2) [ 0.00791956 -0.00449494  0.00599633]
TukeyBiweight(1.2) [ 0.00779914 -0.00449494  0.00582663]
TukeyBiweight(1) [ 0.00776729 -0.00449494  0.00578071]
TukeyBiweight(.8) [ 0.0077379  -0.00449494  0.00573725]
TukeyBiweight(.5) [ 0.00781944 -0.00449494  0.00583018]
TukeyBiweight(.25) [ 0.00838992 -0.00449494  0.00660273]


## look for a good filter

In [59]:
def bw(sig, N, Wn):
    sos = butter(N, Wn, fs=20, output='sos')
    return sosfilt(sos, sig)

In [ ]:
%%capture
filters = [lambda x: median_filter(x, 10), lambda x: median_filter(x, 20), lambda x: median_filter(x, 50), lambda x: median_filter(x, 100),
          lambda x: uniform_filter1d(x, 10), lambda x: uniform_filter1d(x, 20), lambda x: uniform_filter1d(x, 50), lambda x: uniform_filter1d(x, 100),
           lambda x: bw(x, 2, .3), lambda x: bw(x, 2, 1), lambda x: bw(x, 2, 3),
           lambda x: bw(x, 4, .3), lambda x: bw(x, 4, 1), lambda x: bw(x, 4, 3)]
keys = ["med10", "med20", "med50", "med100", "mean10", "mean20", "mean50", "mean100",
        "bw2_.3", "bw2_1", "bw2_3", "bw4_.3", "bw4_1", "bw4_3"]
res2 = {}
for k, f in zip(keys, filters):
    res2[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [f]*N, [TukeyBiweight(1)]*N))

In [66]:
print("Method      RMSE       bias     std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res2[k]]).mean(0)[...,1:].mean((1,2)))

Method      RMSE       bias     std_of_diff
med10 [ 0.00764844 -0.00449494  0.00560843]
med20 [ 0.00763152 -0.00449494  0.00558691]
med50 [ 0.00762363 -0.00449494  0.00557796]
med100 [ 0.00761339 -0.00449494  0.0055639 ]
mean10 [ 0.00766168 -0.00449494  0.0056293 ]
mean20 [ 0.00763661 -0.00449494  0.00559566]
mean50 [ 0.00761622 -0.00449494  0.00556811]
mean100 [ 0.00760842 -0.00449494  0.00555757]
bw2_.3 [ 0.00762732 -0.00449494  0.00558347]
bw2_1 [ 0.00766725 -0.00449494  0.00563687]
bw2_3 [ 0.00772482 -0.00449494  0.00571694]
bw4_.3 [ 0.00762817 -0.00449494  0.00558525]
bw4_1 [ 0.00766491 -0.00449494  0.00563372]
bw4_3 [ 0.00772351 -0.00449494  0.00571514]


### filter both traces to estimate regression coefs

In [ ]:
%%capture
filters = [lambda x: median_filter(x, 10), lambda x: median_filter(x, 20), lambda x: median_filter(x, 50), lambda x: median_filter(x, 100),
          lambda x: uniform_filter1d(x, 10), lambda x: uniform_filter1d(x, 20), lambda x: uniform_filter1d(x, 25),
           lambda x: uniform_filter1d(x, 30), lambda x: uniform_filter1d(x, 40), lambda x: uniform_filter1d(x, 50), lambda x: uniform_filter1d(x, 100),
           lambda x: bw(x, 2, .1), lambda x: bw(x, 2, .3), lambda x: bw(x, 2, 1), lambda x: bw(x, 2, 3),
           lambda x: bw(x, 4, .1), lambda x: bw(x, 4, .3), lambda x: bw(x, 4, 1), lambda x: bw(x, 4, 3),
           lambda x: bw(x, 6, .1)]
keys = ["med10", "med20", "med50", "med100", "mean10", "mean20", "mean25", "mean30", "mean40", "mean50", "mean100",
        "bw2_.1", "bw2_.3", "bw2_.5", "bw2_1", "bw2_3", "bw4_.1", "bw4_.3", "bw4_.5", "bw4_1", "bw4_3", "bw6_.1", "bw6_.3", "bw6_.5"]
res3 = {}
for k, f in zip(keys, filters):
    res3[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [f]*N, [TukeyBiweight(1)]*N, [True]*N))

In [113]:
%%capture
filters = [lambda x: uniform_filter1d(x, 25), lambda x: uniform_filter1d(x, 30), lambda x: uniform_filter1d(x, 40)]
keys = ["mean25", "mean30", "mean40"]
for k, f in zip(keys, filters):
    res3[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [f]*N, [TukeyBiweight(1)]*N, [True]*N))

In [114]:
keys = ["med10", "med20", "med50", "med100", "mean10", "mean20", "mean25", "mean30", "mean40", "mean50", "mean100",
        "bw2_.1", "bw2_.3", "bw2_.5", "bw2_1", "bw2_3", "bw4_.1", "bw4_.3", "bw4_.5", "bw4_1", "bw4_3", "bw6_.1", "bw6_.3", "bw6_.5"]

In [115]:
print("Method      RMSE       bias     std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res3[k]]).mean(0)[...,1:].mean((1,2)))

Method      RMSE       bias     std_of_diff
med10 [ 0.00766165 -0.00449494  0.00562709]
med20 [ 0.00761504 -0.00449494  0.00556521]
med50 [ 0.00759686 -0.00449494  0.00554365]
med100 [ 0.00762321 -0.00449494  0.00558331]
mean10 [ 0.00760264 -0.00449494  0.00554888]
mean20 [ 0.00758509 -0.00449494  0.00552581]
mean25 [ 0.00758481 -0.00449494  0.00552589]
mean30 [ 0.00758622 -0.00449494  0.00552833]
mean40 [ 0.00759213 -0.00449494  0.00553733]
mean50 [ 0.00759894 -0.00449494  0.00554777]
mean100 [ 0.00764128 -0.00449494  0.00561139]
bw2_.1 [ 0.00763436 -0.00449494  0.00559698]
bw2_.3 [ 0.00758191 -0.00449494  0.0055218 ]
bw2_.5 [ 0.00763436 -0.00449494  0.00559698]
bw2_1 [ 0.00760632 -0.00449494  0.00555363]
bw2_3 [ 0.00767973 -0.00449494  0.00565617]
bw4_.1 [ 0.00763794 -0.00449494  0.00560137]
bw4_.3 [ 0.00758095 -0.00449494  0.00552025]
bw4_.5 [ 0.00763794 -0.00449494  0.00560137]
bw4_1 [ 0.00760273 -0.00449494  0.00554882]
bw4_3 [ 0.00767714 -0.00449494  0.00565255]
bw6_.1 [ 0.007646

### moving average works well, try some more windowing function

In [94]:
def smooth_signal(source, window_len=10, window='flat'):
    """Smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    Args :  source (arr) = the input signal 
            window_len (int) = the dimension of the smoothing window; should be an odd integer
            window (str) = the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    Returns : sink = the smoothed signal
    
    Code taken from (https://scipy-cookbook.readthedocs.io/items/SignalSmooth.html)
    """

    if source.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if source.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len < 3:
        return source

    if window not in ('flat', 'hanning', 'hamming', 'bartlett', 'blackman'):
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    # s = np.r_[source[window_len-1:0:-1], source, source[-2:-window_len-1:-1]]
    s = np.r_[source[(window_len)//2:0:-1], source, source[-2:-window_len//2-1:-1]]

    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.'+window+'(window_len)')

    sink = np.convolve(w/w.sum(), s, mode='valid')
    return sink

In [95]:
%%capture
filters = [lambda x: smooth_signal(x, 20, 'hanning'), lambda x: smooth_signal(x, 50, 'hanning'),
          lambda x: smooth_signal(x, 20, 'hamming'), lambda x: smooth_signal(x, 50, 'hamming'), 
          lambda x: smooth_signal(x, 20, 'bartlett'), lambda x: smooth_signal(x, 50, 'bartlett'), 
          lambda x: smooth_signal(x, 20, 'blackman'), lambda x: smooth_signal(x, 50, 'blackman'), ]
keys = ["han20", "han50", "ham20", "ham50", "bar20", "bar50", "bla20", "bla50"]
res4 = {}
for k, f in zip(keys, filters):
    res4[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [f]*N, [TukeyBiweight(1)]*N, [True]*N))

In [96]:
print("Method      RMSE       bias     std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res4[k]]).mean(0)[...,1:].mean((1,2)))

Method      RMSE       bias     std_of_diff
han20 [ 0.00759225 -0.00449494  0.00553483]
han50 [ 0.00758507 -0.00449494  0.00552676]
ham20 [ 0.00758878 -0.00449494  0.00553023]
ham50 [ 0.00758729 -0.00449494  0.00553004]
bar20 [ 0.00758858 -0.00449494  0.00552994]
bar50 [ 0.0075875  -0.00449494  0.00553036]
bla20 [ 0.00759813 -0.00449494  0.00554277]
bla50 [ 0.00758381 -0.00449494  0.0055247 ]


## check that the M estimator we found without any filtering still performs best with filtering

In [109]:
%%capture
N = len(parents)
Ms = [None, HuberT(.5), HuberT(.25), HuberT(.1),
      TukeyBiweight(2), TukeyBiweight(1.2), TukeyBiweight(1), TukeyBiweight(.8), TukeyBiweight(.5)]
keys = ["OLS", "HuberT(.5)", "HuberT(.25)", "HuberT(.1)",
        "TukeyBiweight(2)", "TukeyBiweight(1.2)", "TukeyBiweight(1)", "TukeyBiweight(.8)", "TukeyBiweight(.5)"]
res5 = {}
for k, m in zip(keys, Ms):
    res5[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [lambda x: bw(x, 2, .3)]*N, [m]*N, [True]*N))

In [110]:
print("Method       RMSE        bias          std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res5[k]]).mean(0)[...,1:].mean((1,2)))

Method       RMSE        bias          std_of_diff
OLS [ 0.00839741 -0.00449494  0.00661027]
HuberT(.5) [ 0.007968   -0.00449494  0.00606435]
HuberT(.25) [ 0.00794446 -0.00449494  0.00603281]
HuberT(.1) [ 0.00793756 -0.00449494  0.00602352]
TukeyBiweight(2) [ 0.00772541 -0.00449494  0.00573469]
TukeyBiweight(1.2) [ 0.00760775 -0.00449494  0.00556167]
TukeyBiweight(1) [ 0.00758191 -0.00449494  0.0055218 ]
TukeyBiweight(.8) [ 0.0075672  -0.00449494  0.00549671]
TukeyBiweight(.5) [ 0.00777972 -0.00449494  0.00577042]


In [111]:
%%capture
N = len(parents)
Ms = [TukeyBiweight(1.2), TukeyBiweight(1), TukeyBiweight(.8), TukeyBiweight(.5)]
keys = ["TukeyBiweight(1.2)", "TukeyBiweight(1)", "TukeyBiweight(.8)", "TukeyBiweight(.5)"]
res6 = {}
for k, m in zip(keys, Ms):
    res6[k] = ThreadPool(N).starmap(evaluate, zip(range(N), [lambda x: uniform_filter1d(x, 20)]*N, [m]*N, [True]*N))

In [112]:
print("Method       RMSE        bias          std_of_diff")
for k in keys:
    print(k, np.concatenate([np.squeeze(r[0]) for r in res6[k]]).mean(0)[...,1:].mean((1,2)))

Method       RMSE        bias          std_of_diff
TukeyBiweight(1.2) [ 0.00761396 -0.00449494  0.00556976]
TukeyBiweight(1) [ 0.00758509 -0.00449494  0.00552581]
TukeyBiweight(.8) [ 0.0075654  -0.00449494  0.00549455]
TukeyBiweight(.5) [ 0.00777234 -0.00449494  0.00575991]


In [ ]:
def fit(self, maxiter=50, tol=1e-8, scale_est='mad', init=None, cov='H1',
            update_scale=True, conv='dev', start_params=None)

In [42]:
def motion_correct0(dff, filt=lambda x: median_filter(x, 10), M=TukeyBiweight(1), zero_mean=True):
    motion = dff[0]
    if filt is not None:
        motion = filt(motion)
    if M is not None:
        motion = np.maximum([RLM(d, motion, M=M).fit(tol=1e-5).params for d in dff], 0) * motion  # should we rather use robust regression?
    else:
        # motion = LinearRegression().fit(motion[:,None], dff.T).coef_ * motion
        motion = LinearRegression(fit_intercept=False, positive=True
                                 ).fit(motion[:,None], dff.T).predict(motion[:,None]).T
    dff_mc = dff - motion
    if zero_mean:
        dff_mc += motion.mean(-1)[...,None]
    return dff_mc, motion

In [43]:
def evaluate0(row, filt=lambda x: x, M=TukeyBiweight(1), zero_mean=True, pres=("bright",)):
    perf, perf_motion = [], []

    parent = parents[row]
    for exp_id, nwb_file in enumerate(sorted(glob.glob(f"{parent}/*/*.nwb"))):
        # print(f"Processing NWB file: {nwb_file}")
        with h5py.File(Path(nwb_file).parent / 'groundtruth.h5') as file:
            dFF_gt = [file[f'fiber{f}/dff'][:] for f in (0,1)]
            motion_gt = [file[f'fiber{f}/motion'][:] for f in (0,1)]
        tmp = []
        tmpm = []
        for pre in pres:
            dFFpre = results[row][2][exp_id, {"poly": 0, "exp": 1, "bright": 2}[pre]]
            dFF, motion = np.transpose([motion_correct0(dff, filt, M, zero_mean) for dff in dFFpre], (1,0,2,3))
            tmp.append([RMSE(dFF, dFF_gt), bias(dFF, dFF_gt), std_of_diff(dFF, dFF_gt)])
            tmpm.append(np.array([[np.corrcoef(m,g)[0,1] for m,g in zip(motion[f], motion_gt[f])] for f in (0,1)]))
        perf.append(tmp)
        perf_motion.append(tmpm)
    return perf, perf_motion

In [44]:
%%capture
tic = -time()
res0 = ThreadPool(N).map(evaluate0, range(N))
tic += time()
print(tic)

In [45]:
# fit(tol=1e-5)
tic, np.concatenate([np.squeeze(r[0]) for r in res0]).mean(0)[...,1:].mean((1,2))

(274.697473526001, array([ 0.00776729, -0.00449494,  0.00578071]))

In [41]:
# fit(maxiter=30)
tic, np.concatenate([np.squeeze(r[0]) for r in res0]).mean(0)[...,1:].mean((1,2))

(173.51955890655518, array([ 0.0077686 , -0.00449494,  0.00578242]))

In [37]:
# fit() w/o params
tic, np.concatenate([np.squeeze(r[0]) for r in res0]).mean(0)[...,1:].mean((1,2))

(275.6970102787018, array([ 0.00776729, -0.00449494,  0.00578071]))